In [66]:
import gtdynamics as gtd
from gtdynamics import ContactGoal, PointOnLink, Slice, Interval
from gtsam import Pose3, Point3
import pandas
import plotly.express as px
import pybullet as p
import gtsam
import roboplot  # Be sure to install package with `pip install -U roboplot`

ModuleNotFoundError: No module named 'roboplot'

In [67]:
robot = gtd.CreateRobotFromFile(gtd.URDF_PATH + "/humanoid/nao.urdf");
# robot = gtd.CreateRobotFromFile(gtd.URDF_PATH + "/quad_climb_robot.urdf");
# roboplot.plot_robot(robot, "body")

RuntimeError: PoseRelativeToGraph error: multiple incoming edges to current vertex [LAnklePitch].

In [49]:
link_names = [(link.id(), link.name()) for link in robot.links()]
link_names.sort()
print(link_names)

[(0, 'body'), (1, 'hip2'), (2, 'upper2'), (3, 'lower2'), (4, 'grab2'), (5, 'hip3'), (6, 'upper3'), (7, 'lower3'), (8, 'grab3'), (9, 'hip0'), (10, 'upper0'), (11, 'lower0'), (12, 'grab0'), (13, 'hip1'), (14, 'upper1'), (15, 'lower1'), (16, 'grab1')]


In [59]:
# # Figure out which feet are front, back:
# for name in ["r_wrist","l_wrist","l_ankle","r_ankle"]:
#     print(name, robot.link(name).wTcom().translation())
# print(dir(robot.link("r_gripper")))
# print(robot.link("r_gripper").lTcom())

for name in ["grab0","grab1","grab2","grab3"]:
    print(name, robot.link(name).bMcom().translation())

grab0 [ 0.44521309  0.40246222 -0.02307521]
grab1 [-0.44521242  0.402463   -0.02307521]
grab2 [ 0.44521175 -0.40246379 -0.02307521]
grab3 [-0.44521242 -0.402463   -0.02307521]


In [63]:
# r_hand = "grab0" # leg 1 = red
# # r_wrist = "12"
# l_hand = "grab1" # leg 2 = purple
# l_wrist = "13"

# r_foot = "grab2" # leg 3 = green
# r_ankle = "14"
# l_foot = "grab3" # leg 4 = blue
# l_ankle = "14"

# LH = PointOnLink(robot.link(l_hand))
# LF = PointOnLink(robot.link(l_foot))
# RH = PointOnLink(robot.link(r_hand))
# RF = PointOnLink(robot.link(r_foot))

# # Test trivial inverse kinematics 
# contact_goals = [
#     ContactGoal(LH, robot.link(l_hand).wTcom().translation()),
#     ContactGoal(LF, robot.link(l_ankle).wTcom().translation()),
#     ContactGoal(RH, robot.link(r_hand).wTcom().translation()),
#     ContactGoal(RF, robot.link(r_ankle).wTcom().translation()),
#     ]
# print(contact_goals)

# mapping = {
#     "LH":robot.link("grab1"),
#     "LF":robot.link("grab3"),
#     "RF":robot.link("grab2"),
#     "RH":robot.link("grab0"),
#     }
# print([(key,link.id()) for key,link in mapping.items()])

# feet
contact_in_com =(0.14, 0, 0)
LH = PointOnLink(robot.link("lower1"), contact_in_com)
LF = PointOnLink(robot.link("lower3"), contact_in_com)
RF = PointOnLink(robot.link("lower2"), contact_in_com)
RH = PointOnLink(robot.link("lower0"), contact_in_com)

# establish contact/goal pairs
contact_goals = [
    ContactGoal(LH, [-0.44521242,  0.402463,   -0.02307521]),
    ContactGoal(LF, [-0.44521242, -0.402463,   -0.02307521]),
    ContactGoal(RF, [ 0.44521175, -0.40246379, -0.02307521]),
    ContactGoal(RH, [ 0.44521309,  0.40246222, -0.02307521])
    ]
print(contact_goals)

[{{lower1, [0.14    0    0]}, [ -0.445212   0.402463 -0.0230752]}, {{lower3, [0.14    0    0]}, [ -0.445212  -0.402463 -0.0230752]}, {{lower2, [0.14    0    0]}, [  0.445212  -0.402464 -0.0230752]}, {{lower0, [0.14    0    0]}, [  0.445213   0.402462 -0.0230752]}]


In [64]:
kinematics = gtd.Kinematics(robot)
result = kinematics.inverseSlice(Slice(4), contact_goals)

TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. gtdynamics.gtdynamics.Kinematics()

Invoked with: 
LINKS:
body, id=0:
	com pose:  0 -0  0, 0 0 0
	joints: 10 11 8 9 
hip2, id=1:
	com pose:       0      -0 -0.7075,  0.15 -0.15     0
	joints: 10 4 
upper2, id=2:
	com pose: -3.17477e-17       0.7075      2.43409,  0.236637 -0.224092 0.0974904
	joints: 4 5 
lower2, id=3:
	com pose: -1.12924e-16      -0.8625      2.43409,  0.242507 -0.229111  0.288308
	joints: 5 14 
grab2, id=4:
	com pose: -1.12924e-16      -0.8625      2.43409,   0.445212  -0.402464 -0.0230752
	joints: 14 
hip3, id=5:
	com pose:      0     -0 0.7075, -0.15 -0.15     0
	joints: 11 6 
upper3, id=6:
	com pose: -8.56822e-18       0.7075       0.7075, -0.236638 -0.224092 0.0974904
	joints: 6 7 
lower3, id=7:
	com pose: 3.13575e-17     -0.8625      0.7075, -0.242507 -0.229111  0.288308
	joints: 7 15 
grab3, id=8:
	com pose: 3.13575e-17     -0.8625      0.7075,  -0.4452

In [53]:
for goal in contact_goals:
    print(goal.link().name(), goal.satisfied(result,k=4,tol=1e-3))

NameError: name 'contact_goals' is not defined

In [56]:
robot.link("grab0").wTcom().translation()

AttributeError: 'gtdynamics.gtdynamics.Link' object has no attribute 'wTcom'

In [57]:
import numpy as np
points = np.asarray([robot.link("l_gripper").wTcom().translation(), 
                     robot.link("l_ankle").wTcom().translation(), 
                     robot.link("r_gripper").wTcom().translation(), 
                     robot.link("r_ankle").wTcom().translation()])

RuntimeError: no link named l_gripper

In [58]:
from mpl_toolkits import mplot3d


%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt



fig = plt.figure()
ax = plt.axes(projection='3d')


ax.scatter3D(points[:, 0], points[:, 1], points[:, 2]);

ModuleNotFoundError: No module named 'mpl_toolkits'

In [123]:
robot.link("l_ankle").wTcom().translation()

array([ 0.02541975,  0.05330005, -0.32029059])

In [139]:
import unittest

class TestRobot(unittest.TestCase):
    
    def testInverseKinematicsHands(self):
        robot = gtd.CreateRobotFromFile(gtd.URDF_PATH + "/humanoid/nao.urdf")
        contact_goals = [
            ContactGoal(LH, [.01, .113, .26]),
            ContactGoal(LF, robot.link("l_ankle").wTcom().translation()),
            ContactGoal(RH, [.01, -.113, .26]),
            ContactGoal(RF, robot.link("r_ankle").wTcom().translation()),
        ]
        kinematics = gtd.Kinematics(robot)
        result = kinematics.inverseSlice(Slice(4), contact_goals)
        for goal in contact_goals:
            self.assertTrue(goal.satisfied(result,k=4,tol=1e-3))
            
    def testInverseKinematicsAll(self):
        robot = gtd.CreateRobotFromFile(gtd.URDF_PATH + "/humanoid/nao.urdf")
        contact_goals = [
            ContactGoal(LH, [.01, .113, .26]),
            ContactGoal(LF, [0.025, 0.05, -0.3]),
            ContactGoal(RH, [.01, -.113, .26]),
            ContactGoal(RF, [0.025, -0.05, -0.3]),
        ]
        kinematics = gtd.Kinematics(robot)
        result = kinematics.inverseSlice(Slice(4), contact_goals)
        for goal in contact_goals:
            self.assertTrue(goal.satisfied(result,k=4,tol=1e-3))
            
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.775s

OK


In [47]:
_link_name_to_index = {p.getBodyInfo(climb_robot)[0].decode('UTF-8'):-1,}
        
for _id in range(p.getNumJoints(climb_robot)):
    _name = p.getJointInfo(pybullet_robot, _id)[12].decode('UTF-8')
    _link_name_to_index[_name] = _id

NameError: name 'pybullet_robot' is not defined

In [46]:
_link_name_to_index

NameError: name '_link_name_to_index' is not defined